In [1]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install rq
# !{sys.executable} -m pip install numpy

In [2]:
# Import packages for use
import pandas as pd
import requests as rq
from io import BytesIO
import re
import numpy as np

# importing .xlsx files from github
sheets = [1,2,3,4,5,6,7,8,9,10,11,12,13] # each county one sheet

data_rq1 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/raw/e79f6b7bdcdc20c12a65c19373776eb25000edae/GR14_Bewerberdatei_SN_KS_LK_Gem.xlsx').content
data_rq2 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/raw/e79f6b7bdcdc20c12a65c19373776eb25000edae/GR19_Bewerber_Stand_Juli2020_Final.xlsx').content

sheets_dict = pd.read_excel(BytesIO(data_rq1), sheet_name=sheets, header=2)
sheets_dict_2 = pd.read_excel(BytesIO(data_rq2), sheet_name=sheets, header=2)

In [3]:
# get append datasets
data_appended_1 = pd.DataFrame()
data_appended_2 = pd.DataFrame()
for name, sheet in sheets_dict.items():
    data_appended_1 = data_appended_1.append(sheet)
    data_appended_1['year'] = 2014
for name, sheet in sheets_dict_2.items():
    data_appended_2 = data_appended_2.append(sheet)
    data_appended_2['year'] = 2019
data_appended = data_appended_1.append(data_appended_2)
data_appended.reset_index(inplace=True, drop=True)
data_appended

,BZ_BZID,EBENE,ORTNR,ORTNAME,EBENE 1,ORTNR 1,ORTNAME 1,KW_LISNR,Kurzbezeichnung,Langbezeichnung,Anzahl Bewerber,darunter Frauen,year,Unnamed: 12
0,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",1,DIE LINKE,DIE LINKE,40,11,2014,NaN
1,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",2,CDU,Christlich Demokratische Union Deutschlands,49,11,2014,NaN
2,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",3,SPD,Sozialdemokratische Partei Deutschlands,63,19,2014,NaN
3,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",4,FDP,Freie Demokratische Partei,78,18,2014,NaN
4,GR14,SN,14,Sachsen,KS,14511000,"Chemnitz, Stadt",5,GRÜNE,BÜNDNIS 90/DIE GRÜNEN,64,26,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,GR19,LK,14730,Nordsachsen,GE,14730340,Wiedemar,2,SPD,Sozialdemokratische Partei Deutschlands,7,2,2019,NaN
4963,GR19,LK,14730,Nordsachsen,GE,14730340,Wiedemar,3,Freie Wähler,Freie Wählervereinigung Wiedemar,14,6,2019,NaN
4964,GR19,LK,14730,Nordsachsen,GE,14730340,Wiedemar,4,Sächsicher Nordzipfel,"Heimatverein ""Sächsicher Nordzipfel""",4,3,2019,NaN
4965,GR19,LK,14730,Nordsachsen,GE,14730360,Zschepplin,1,CDU,Christlich Demokratische Union Deutschlands,14,3,2019,NaN


In [4]:
# rename columns and filter
data_appended = data_appended[['ORTNAME 1', 'ORTNR 1', 'Kurzbezeichnung', 'Anzahl Bewerber', 'darunter Frauen', 'year']]
data_appended.rename({'ORTNAME 1':'city', 'Kurzbezeichnung': 'party', 'ORTNR 1':'city_id'}, axis=1, inplace=True)
data_appended['state'] = 'SN'

data_appended = data_appended[(data_appended['party'] == 'DIE LINKE') | (data_appended['party'] == 'CDU') | (data_appended['party'] == 'GRÜNE')| (data_appended['party'] == 'SPD')  | (data_appended['party'] == 'FDP') | (data_appended['party'] == 'AfD')]
data_appended['Prozent Frauen'] = data_appended['darunter Frauen']/data_appended['Anzahl Bewerber'] * 100
data_appended.head()

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,city,city_id,party,Anzahl Bewerber,darunter Frauen,year,state,Prozent Frauen
0,"Chemnitz, Stadt",14511000,DIE LINKE,40,11,2014,SN,27.500000
1,"Chemnitz, Stadt",14511000,CDU,49,11,2014,SN,22.448980
2,"Chemnitz, Stadt",14511000,SPD,63,19,2014,SN,30.158730
3,"Chemnitz, Stadt",14511000,FDP,78,18,2014,SN,23.076923
4,"Chemnitz, Stadt",14511000,GRÜNE,64,26,2014,SN,40.625000


In [5]:
data_appended = data_appended[~data_appended['city'].str.contains('Wahlkreis')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Leipzig$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Bautzen$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Meißen$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Görlitz$')]
data_appended = data_appended[~data_appended['city'].str.contains(r'Zwickau$')]

In [6]:
# merge plz to city_id 
data_org = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/AuszugGV4QAktuell.csv', delimiter=';', dtype={'5':'string', 'plz':'string'})
data_org = data_org.dropna(thresh=8).reset_index(drop=True)
data_org.loc[data_org['Land'] ==4, 'bl_kuerzel'] = 'HB'
data_org.loc[data_org['Land'] ==7, 'bl_kuerzel'] = 'RP'
data_org.loc[data_org['Land'] ==8, 'bl_kuerzel'] = 'BW'
data_org.loc[data_org['Land'] ==9, 'bl_kuerzel'] = 'BY'
data_org.loc[data_org['Land'] ==10, 'bl_kuerzel'] = 'SL'
data_org.loc[data_org['Land'] ==12, 'bl_kuerzel'] = 'BB'
data_org.loc[data_org['Land'] ==13, 'bl_kuerzel'] = 'MV'
data_org.loc[data_org['Land'] ==14, 'bl_kuerzel'] = 'SN'
data_org.loc[data_org['Land'] ==15, 'bl_kuerzel'] = 'SA'
data_org.loc[data_org['Land'] ==16, 'bl_kuerzel'] = 'TH'
data_org['Gemeindename'] = data_org['Gemeindename'].str.split(',', expand=True)[0]
# for string merge
data_org['Gemeindename'] = data_org['Gemeindename'].str.replace('ö','o').str.replace('ä', 'a').str.replace('ü', 'u').str.replace('ß', 'ss').str.replace('Ü', 'U').str.replace('Ä', 'A').str.replace('Ö', 'O')
data_org = data_org.rename({'Gemeindename':'city'},axis=1)

In [7]:
data_org.rename({'ags':'city_id'}, axis=1, inplace=True)
data_org_sachsen = data_org[data_org['Land'] == 14]
data_org_sachsen = data_org_sachsen[['plz', 'city_id', 'Land', 'bl_kuerzel']]
data_appended = pd.merge(data_org_sachsen, data_appended, on=['city_id'], how='right', validate='1:m')

In [8]:
# import BW data
data_rq3 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/blob/main/GW2014_Gemeindetabelle_Bewerber_Gew%C3%A4hlte_nach_Geschlecht.xlsx?raw=true').content
data_rq4 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/blob/main/GW2019_Gemeindetabelle_Bewerbungen01.xlsx?raw=true').content
data_rq5 = rq.get(r'https://github.com/mariusgruenewald/pol_viol/blob/main/GW2019_Gemeindetabelle_Bewerbungen02.xlsx?raw=true').content

data_bw_2014 = pd.read_excel(BytesIO(data_rq3), header=4)
data_bw_2019_1 = pd.read_excel(BytesIO(data_rq4), header=2)
data_bw_2019_2 = pd.read_excel(BytesIO(data_rq5), header=2)

In [9]:
# for merging with BW-Data generate city_id consisting of RB + Kreis + Gemeinde. Do not use general ags
data_org_bw = data_org.copy()
data_org_bw['city_id_bw'] = data_org_bw['2'].astype(int).astype(str) + data_org_bw['3'].astype(int).astype(str) + data_org_bw['5']
data_org_bw = data_org_bw[data_org_bw["1"] == 8]
data_org_bw['city_id_bw'] = data_org_bw['city_id_bw'].astype(float)
data_org_bw

,Land,city,Fläche,insgesamt,männlich,weiblich,density,plz,Längengrad,Breitengrad,...,Schlüssel_verst,Verstädterung,1,2,3,4,5,city_id,bl_kuerzel,city_id_bw
5203,8,Stuttgart,"207,33",635911.0,317794.0,318117.0,3067.0,70173,"9,177734","48,775817",...,1.0,dicht besiedelt,8.0,1.0,11.0,0.0,000,8111000.0,BW,111000.0
5204,8,Boblingen,"39,04",50161.0,24805.0,25356.0,1285.0,71032,"9,009663","48,685296",...,1.0,dicht besiedelt,8.0,1.0,15.0,3.0,003,8115003.0,BW,115003.0
5205,8,Leonberg,"48,73",48848.0,24411.0,24437.0,1002.0,71229,"9,016776","48,800497",...,2.0,mittlere Besiedlungsdichte,8.0,1.0,15.0,28.0,028,8115028.0,BW,115028.0
5206,8,Magstadt,"19,12",9711.0,4946.0,4765.0,508.0,71106,"8,962269","48,742432",...,2.0,mittlere Besiedlungsdichte,8.0,1.0,15.0,29.0,029,8115029.0,BW,115029.0
5207,8,Renningen,"31,15",18487.0,9195.0,9292.0,593.0,71272,"8,936373","48,765313",...,2.0,mittlere Besiedlungsdichte,8.0,1.0,15.0,41.0,041,8115041.0,BW,115041.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6301,8,Krauchenwies,"44,65",5001.0,2515.0,2486.0,112.0,72505,"9,248681","48,018687",...,3.0,gering besiedelt,8.0,4.0,37.0,5006.0,065,8437065.0,BW,437065.0
6302,8,Sigmaringen,"92,84",17125.0,8557.0,8568.0,184.0,72488,"9,214788","48,086623",...,2.0,mittlere Besiedlungsdichte,8.0,4.0,37.0,5006.0,104,8437104.0,BW,437104.0
6303,8,Sigmaringendorf,"12,47",3646.0,1851.0,1795.0,292.0,72517,"9,261616","48,067985",...,3.0,gering besiedelt,8.0,4.0,37.0,5006.0,105,8437105.0,BW,437105.0
6304,8,Schwenningen,"19,34",1670.0,840.0,830.0,86.0,72477,"8,997429","48,10473",...,3.0,gering besiedelt,8.0,4.0,37.0,5007.0,102,8437102.0,BW,437102.0


In [10]:
# get data into long format
data_bw_2014.rename({'Unnamed: 1':'city_id_bw', 'Unnamed: 2':'city', 'Unnamed: 3':'category', 'Unnamed: 4': 'unit'},axis=1, inplace=True)
data_bw_2014 = data_bw_2014[['city_id_bw', 'city', 'category', 'unit', 'CDU', 'SPD', 'FDP', 'GRÜNE', 'AfD', 'DIE LINKE']]
data_bw_2014.dropna(inplace=True)
data_long = data_bw_2014[['city_id_bw', 'city', 'category', 'unit', 'CDU']]
data_long['party'] = 'CDU'
data_long.rename({'CDU':'value'},axis=1, inplace=True)

list_cols = ['city_id_bw', 'city', 'category', 'unit']
for party in ('SPD', 'FDP', 'GRÜNE', 'AfD', 'DIE LINKE'):
    list_cols.append(party)
    data_temp = data_bw_2014[list_cols]
    data_temp['party'] = party
    data_temp.rename({party:'value'},axis=1, inplace=True)
    list_cols = ['city_id_bw', 'city', 'category', 'unit']
    data_long = data_long.append(data_temp)
    
del data_temp
data_long.reset_index(drop=True, inplace=True)

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [11]:
# adjust format to match above format
data_long = data_long[(data_long['category'] != 'Gewählte') & (data_long['category'] != 'Gewählte Frauen')]
data_long['type'] = 'Anzahl Bewerber'
data_long.loc[(data_long['category'] == 'Bewerberinnen') & (data_long['unit'] == 'Anzahl'), 'type'] = 'darunter Frauen'
data_long.loc[(data_long['category'] == 'Bewerberinnen') & (data_long['unit'] == '%'), 'type'] = 'Prozent Frauen'
data_long.drop(['category', 'unit'], axis=1)
data_long_2014 = data_long.pivot(index=['city_id_bw', 'city', 'party'], columns=['type'], values='value')
data_long_2014['year'] = 2014
data_long_2014.reset_index(inplace=True)
data_long_2014['state'] = 'BW'

In [12]:
# renaming and filtering
data_bw_2019_1.rename({'Davon':'CDU', 'Unnamed: 8':'GRÜNE', 'Unnamed: 9':'SPD', 'Unnamed: 10':'FDP', 'Unnamed: 11':'AfD', 'Unnamed: 2':'city_id_bw'}, axis=1, inplace=True)
data_bw_2019_1 = data_bw_2019_1[['city_id_bw', 'Gemeinde', 'Bezeichnung', 'Ein-\nheit', 'CDU', 'SPD', 'FDP', 'GRÜNE', 'AfD']]
data_bw_2019_1.dropna(inplace=True)
data_bw_2019_2.rename({'Unnamed: 9':'DIE LINKE', 'Unnamed: 2': 'city_id_bw'}, axis=1, inplace=True)
data_bw_2019_2 = data_bw_2019_2[['city_id_bw', 'Gemeinde', 'Bezeichnung', 'Ein-\nheit', 'DIE LINKE']]
data_bw_2019_2.dropna(inplace=True)
data_bw_2019 = pd.merge(data_bw_2019_1, data_bw_2019_2, on=['city_id_bw', 'Gemeinde', 'Bezeichnung', 'Ein-\nheit'], how='inner', validate='1:1')
data_bw_2019

,city_id_bw,Gemeinde,Bezeichnung,Ein-\nheit,CDU,SPD,FDP,GRÜNE,AfD,DIE LINKE
0,111000,"Stuttgart, Landeshauptstadt",Bewerberinnen und Bewerber,Anzahl,60,60,60,60,60,60
1,111000,"Stuttgart, Landeshauptstadt",Bewerberinnen,Anzahl,25,30,18,30,16,29
2,111000,"Stuttgart, Landeshauptstadt",Bewerberinnenanteil,%,41.666667,50,30,50,26.666667,48.333333
3,111000,"Stuttgart, Landeshauptstadt",Unionsbürger,Anzahl,1,0,2,4,0,0
4,111000,"Stuttgart, Landeshauptstadt",Unionsbürgeranteil,%,1.666667,0,3.333333,6.666667,0,0
...,...,...,...,...,...,...,...,...,...,...
5500,437124,Herdwangen-Schönach,Bewerberinnen und Bewerber,Anzahl,7,0,0,0,0,0
5501,437124,Herdwangen-Schönach,Bewerberinnen,Anzahl,0,0,0,0,0,0
5502,437124,Herdwangen-Schönach,Bewerberinnenanteil,%,0,0,0,0,0,0
5503,437124,Herdwangen-Schönach,Unionsbürger,Anzahl,0,0,0,0,0,0


In [13]:
# same procedure as above for 2014 data
data_bw_2019 = data_bw_2019[(data_bw_2019['Bezeichnung'] != 'Unionsbürger') & (data_bw_2019['Bezeichnung'] != 'Unionsbürgeranteil')]
data_bw_2019.rename({'Gemeinde': 'city', 'Bezeichnung':'category', 'Ein-\nheit':'unit'}, axis=1, inplace=True)
data_long = data_bw_2019[['city_id_bw', 'city', 'category', 'unit', 'CDU']]
data_long['party'] = 'CDU'
data_long.rename({'CDU':'value'},axis=1, inplace=True)

list_cols = ['city_id_bw', 'city', 'category', 'unit']
for party in ('SPD', 'FDP', 'GRÜNE', 'AfD', 'DIE LINKE'):
    list_cols.append(party)
    data_temp = data_bw_2019[list_cols]
    data_temp['party'] = party
    data_temp.rename({party:'value'},axis=1, inplace=True)
    list_cols = ['city_id_bw', 'city', 'category', 'unit']
    data_long = data_long.append(data_temp)
    
del data_temp
data_long.reset_index(drop=True, inplace=True)

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [14]:
# filter and organize data
data_long['type'] = 'Anzahl Bewerber'
data_long.loc[(data_long['category'] == 'Bewerberinnen') & (data_long['unit'] == 'Anzahl'), 'type'] = 'darunter Frauen'
data_long.loc[(data_long['category'] == 'Bewerberinnenanteil') & (data_long['unit'] == '%'), 'type'] = 'Prozent Frauen'
data_long.drop(['category', 'unit'], axis=1)
data_long_2019 = data_long.pivot(index=['city_id_bw', 'city', 'party'], columns=['type'], values='value')
data_long_2019.reset_index(drop=False, inplace=True)
data_long_2019['year'] = 2019

In [15]:
# append bw data
data_bw = data_long_2019.append(data_long_2014)
data_bw.reset_index(drop=True,inplace=True)
data_bw['state'] = 'BW'

In [16]:
data_bw = pd.merge(data_bw, data_org_bw[['city_id_bw', 'plz', 'city_id', 'Land', 'bl_kuerzel']], on='city_id_bw', how='inner', validate='m:1')
data_bw.drop('city_id_bw', inplace=True, axis=1)

In [17]:
# merge SN and BW data
data_so_far = data_bw.append(data_appended)
data_so_far.reset_index(drop=True,inplace=True)

In [18]:
data_so_far

,city,party,Anzahl Bewerber,Prozent Frauen,darunter Frauen,year,state,plz,city_id,Land,bl_kuerzel
0,"Stuttgart, Landeshauptstadt",AfD,60,26.666667,16,2019,BW,70173,8111000.0,8.0,BW
1,"Stuttgart, Landeshauptstadt",CDU,60,41.666667,25,2019,BW,70173,8111000.0,8.0,BW
2,"Stuttgart, Landeshauptstadt",DIE LINKE,60,48.333333,29,2019,BW,70173,8111000.0,8.0,BW
3,"Stuttgart, Landeshauptstadt",FDP,60,30,18,2019,BW,70173,8111000.0,8.0,BW
4,"Stuttgart, Landeshauptstadt",GRÜNE,60,50,30,2019,BW,70173,8111000.0,8.0,BW
...,...,...,...,...,...,...,...,...,...,...,...
15322,Wermsdorf,GRÜNE,1,100.0,1,2019,SN,4779,14730330.0,14.0,SN
15323,Wermsdorf,AfD,3,0.0,0,2019,SN,4779,14730330.0,14.0,SN
15324,Wiedemar,CDU,17,29.411765,5,2019,SN,4509,14730340.0,14.0,SN
15325,Wiedemar,SPD,7,28.571429,2,2019,SN,4509,14730340.0,14.0,SN


In [19]:
data_th_2014 = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/Th_2014.csv', header=3, sep=';')
data_th_2019 = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/Th_2019.csv', header=3, sep=';')
data_th_2014['year'] = 2014
data_th_2019['year'] = 2019
data_th_2014['state'] = 'TH'
data_th_2019['state'] = 'TH'
data_th_2014.dropna(inplace=True)
data_th_2019.dropna(inplace=True)
data_th_2014.rename({'Unnamed: 3': 'darunter Frauen', 'Unnamed: 5': 'female_mem'}, axis=1, inplace=True)
data_th_2019.rename({'Unnamed: 3': 'darunter Frauen', 'Unnamed: 5': 'female_mem'}, axis=1, inplace=True)
data_th = data_th_2019.append(data_th_2014)
data_th['Gemeinde'] = data_th['Gemeinde'].str.split(', ', expand=True)[0]
data_th['city_id_th'] = data_th['Gemeinde'].str.split(r'[\s]', expand=True)[0]
data_th['Gemeinde'] = data_th['Gemeinde'].str.split(r'[[0-9]+[\s]]?', expand=True)[1]
data_th['darunter Frauen'] = data_th['darunter Frauen'].str.replace('-', '0').astype(float)

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\strings\object_array.py:335: FutureWarning: Possible nested set at position 1
  regex = re.compile(pat)


In [20]:
data_th

,Gemeinde,Wahlvorschlag,Anzahl der Bewerber,darunter Frauen,Gewählte Bewerber,female_mem,year,state,city_id_th
1,Erfurt,CDU,49,13.0,10,2,2019,TH,51000
2,Gera,CDU,16,3.0,6,1,2019,TH,52000
3,Jena,CDU,46,10.0,6,3,2019,TH,53000
4,Suhl,CDU,35,7.0,11,2,2019,TH,54000
5,Weimar,CDU,42,9.0,7,1,2019,TH,55000
...,...,...,...,...,...,...,...,...,...
1651,Vollmershain,Sonstige,6,3.0,6,3,2014,TH,77049
1652,Wildenbörten,Sonstige,9,2.0,6,1,2014,TH,77051
1653,Windischleuba,Sonstige,7,2.0,5,1,2014,TH,77052
1654,Ziegelheim,Sonstige,10,1.0,8,1,2014,TH,77055


In [21]:
data_th.dropna(inplace=True)
data_th.reset_index(inplace=True, drop=True)
data_th = data_th[['Gemeinde', 'Wahlvorschlag', 'Anzahl der Bewerber', 'darunter Frauen', 'year', 'state', 'city_id_th']]
data_th.rename({'Gemeinde':'city', 'Wahlvorschlag':'party', 'Anzahl der Bewerber':'Anzahl Bewerber'}, axis=1, inplace=True)
data_th['Anzahl Bewerber'] = data_th['Anzahl Bewerber'].astype(float)
data_th['Prozent Frauen'] = data_th['darunter Frauen']/data_th['Anzahl Bewerber']*100
data_th = data_th[data_th['party'] != 'Sonstige']
data_th

C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

,city,party,Anzahl Bewerber,darunter Frauen,year,state,city_id_th,Prozent Frauen
0,Erfurt,CDU,49.0,13.0,2019,TH,51000,26.530612
1,Gera,CDU,16.0,3.0,2019,TH,52000,18.750000
2,Jena,CDU,46.0,10.0,2019,TH,53000,21.739130
3,Suhl,CDU,35.0,7.0,2019,TH,54000,20.000000
4,Weimar,CDU,42.0,9.0,2019,TH,55000,21.428571
...,...,...,...,...,...,...,...,...
2169,Neuhaus am Rennweg,GRÜNE,1.0,1.0,2014,TH,72013,100.000000
2170,Sonneberg,GRÜNE,7.0,1.0,2014,TH,72018,14.285714
2171,Rudolstadt,GRÜNE,10.0,4.0,2014,TH,73076,40.000000
2172,Saalfeld/Saale,GRÜNE,13.0,5.0,2014,TH,73077,38.461538


In [22]:
data_org_th = data_org.copy()
data_org_th['city_id_th'] = data_org_th['3'].astype(int).astype(str) + data_org_th['5']
data_org_th = data_org_th[data_org_th['Land'] == 16][['plz', 'city_id', 'Land', 'city_id_th', 'bl_kuerzel']]
data_th = pd.merge(data_th, data_org_th, on='city_id_th', how='left', validate='m:1')

In [23]:
data_so_far = data_so_far.append(data_th)
data_so_far.reset_index(drop=True,inplace=True)
data_so_far = data_so_far.drop('city_id_th', axis=1)

In [24]:
data_so_far

,city,party,Anzahl Bewerber,Prozent Frauen,darunter Frauen,year,state,plz,city_id,Land,bl_kuerzel
0,"Stuttgart, Landeshauptstadt",AfD,60,26.666667,16,2019,BW,70173,8111000.0,8.0,BW
1,"Stuttgart, Landeshauptstadt",CDU,60,41.666667,25,2019,BW,70173,8111000.0,8.0,BW
2,"Stuttgart, Landeshauptstadt",DIE LINKE,60,48.333333,29,2019,BW,70173,8111000.0,8.0,BW
3,"Stuttgart, Landeshauptstadt",FDP,60,30,18,2019,BW,70173,8111000.0,8.0,BW
4,"Stuttgart, Landeshauptstadt",GRÜNE,60,50,30,2019,BW,70173,8111000.0,8.0,BW
...,...,...,...,...,...,...,...,...,...,...,...
16848,Neuhaus am Rennweg,GRÜNE,1.0,100.0,1.0,2014,TH,98724,16072013.0,16.0,TH
16849,Sonneberg,GRÜNE,7.0,14.285714,1.0,2014,TH,96515,16072018.0,16.0,TH
16850,Rudolstadt,GRÜNE,10.0,40.0,4.0,2014,TH,7407,16073076.0,16.0,TH
16851,Saalfeld/Saale,GRÜNE,13.0,38.461538,5.0,2014,TH,7318,16073077.0,16.0,TH


In [25]:
# Brandenburg Data

file_list_2014 = ['2014 GV - LK Barnim', '2014 GV - LK Dahme-Spreewald', '2014 GV - LK Elbe-Elster', '2014 GV - LK Havelland', '2014 GV - LK Märkisch-Oderland', '2014 GV - LK Oberhavel', '2014 GV - LK Oberspreewald-Lausitz', 
                  '2014 GV - LK Oder-Spree', '2014 GV - LK Ostprignitz-Ruppin', '2014 GV - LK Potsdam-Mittelmark', '2014 GV - LK Prignitz', '2014 GV - LK Spree-Neiße', '2014 GV - LK Teltow-Fläming', '2014 GV - LK Uckermark']

file_2014_2 = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2014\2014 KT und SVV.xlsx' # sheets [1:5]

file_list_2019 = ['2019 GV - 60 LK Barnim', '2019 GV - 61 LK Dahme-Spreewald', '2019 GV - 62 LK Elbe-Elster', '2019 GV - 63 LK Havelland', '2019 GV - 64 LK Märkisch-Oderland', '2019 GV - 65 LK Oberhavel', '2019 GV - 66 LK Oberspreewald-Lausitz'
                  , '2019 GV - 67 LK Oder-Spree', '2019 GV - 68 LK Ostprignitz-Ruppin', '2019 GV - 69 LK Potsdam-Mittelmark', '2019 GV - 70 LK Prignitz', '2019 GV - 71 LK Spree-Neiße', '2019 GV - 72 LK Teltow-Fläming', 
                 '2019 GV - 73 LK Uckermark']

file_2019_2 = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2019\2019 KT und SVV.xlsx' # sheets [1:5]

In [26]:
# Import county-based data

full_table_14 = pd.DataFrame()
agg_sheet_14 = pd.DataFrame()
for file in file_list_2014:
    path = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2014' + '\\' + file + '.xls'
    
    sheets_dict = pd.read_excel(path, sheet_name=None, skiprows=4)
    for name, sheet in list(sheets_dict.items())[1:]:
        sheet['city'] = name
        full_table_14 = full_table_14.append(sheet)
    
    agg_sheet = pd.read_excel(path, sheet_name=0, skiprows=6)
    agg_sheet_14 = agg_sheet_14.append(agg_sheet)

full_table_19 = pd.DataFrame()
agg_sheet_19 = pd.DataFrame()
for file in file_list_2019:
    path = r'C:\Users\mariu\Documents\pol_viol\pol_viol\Gemeindevertretungen_2019' + '\\' + file + '.xlsx'
    
    sheets_dict = pd.read_excel(path, sheet_name=None, skiprows=4)
    for name, sheet in list(sheets_dict.items())[1:]:
        sheet['city'] = name
        full_table_19 = full_table_19.append(sheet)
    
    # one time is enough since no reform in BB on mun. structure
    #agg_sheet = pd.read_excel(path, sheet_name=0, skiprows=6)
    #agg_sheet_19 = agg_sheet_19.append(agg_sheet)
                
full_table_14.reset_index(inplace=True, drop=True)
full_table_14['year'] = 2014
full_table_19.reset_index(inplace=True, drop=True)
full_table_19['year'] = 2019

agg_sheet_14 = agg_sheet_14[['AGS', 'Name']]
agg_sheet_14.rename({'Name':'city', 'AGS':'city_id_bb'}, inplace=True, axis=1)
agg_sheet_14['city'] = agg_sheet_14['city'].str.split(',', expand=True)[0]
agg_sheet_14

C:\Users\mariu\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,city_id_bb,city
0,60005,Ahrensfelde
1,60020,Bernau bei Berlin
2,60052,Eberswalde
3,60181,Panketal
4,60198,Schorfheide
...,...,...
30,73386,Mark Landin
31,73440,Pinnow
32,73505,Schöneberg
33,73 603,Passow


In [27]:
full_table_19.dropna(thresh=3,axis=1, inplace=True)
full_table_14.dropna(thresh=3,axis=1, inplace=True)

In [28]:
full_table_14.rename({'Unnamed: 0':'party_long', 'Unnamed: 2':'party', 'Unnamed: 3':'Wg_o_P', 'insgesamt':'Anzahl Bewerber',
                     'dar. weiblich':'darunter Frauen'}, axis=1, inplace=True)
full_table_14.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

In [29]:
full_table_19.rename({'Unnamed: 0':'party_long', 'Unnamed: 2':'party', 'Unnamed: 3':'Wg_o_P', 'insgesamt':'Anzahl Bewerber',
                     'dar. weiblich':'darunter Frauen'}, axis=1, inplace=True)
full_table_19.drop(['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 1'], axis=1, inplace=True)

In [30]:
full_table_bb = full_table_14.append(full_table_19)
full_table_bb['city'] = full_table_bb['city'].str.replace(r'(\s[0-9]*)$', '')

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [31]:
# Stadtkreise 2014
sk_2014 = pd.read_excel(file_2014_2, sheet_name=None, skiprows=4)
file_sk_2014 = pd.DataFrame()
for name, sheet in list(sk_2014.items())[1:5]:
    sheet['city'] = name #rename cities
    file_sk_2014 = file_sk_2014.append(sheet) 
    
file_sk_2014['year'] = 2014

In [32]:
# Stadtkreise 2019
sk_2019 = pd.read_excel(file_2019_2, sheet_name=None, skiprows=4)
file_sk_2019 = pd.DataFrame()
for name, sheet in list(sk_2019.items())[1:5]:
    sheet['city'] = name #rename cities
    file_sk_2019 = file_sk_2019.append(sheet)
    
file_sk_2019['year'] = 2019

In [33]:
file_sk_2014.rename({'Unnamed: 0':'party_long', 'Unnamed: 1':'party', 'Unnamed: 2':'Wg_o_P',
                     'Unnamed: 4':'Anzahl Bewerber', 'weibliche\nBewerbende':'darunter Frauen'}, axis=1, inplace=True)
file_sk_2014.drop('Unnamed: 3', axis=1, inplace=True)
file_sk_2014['city'] = file_sk_2014['city'].str.replace('51 - BRB', 'Brandeburg an der Havel').str.replace('52 - CB', 'Cottbus')
file_sk_2014['city'] = file_sk_2014['city'].str.replace('53 - FF', 'Frankfurt (Oder)').str.replace('54 - P', 'Potsdam')

In [34]:
file_sk_2019.rename({'Unnamed: 0':'party_long', 'Unnamed: 1':'party', 'Unnamed: 2':'Wg_o_P',
                     'Unnamed: 4':'Anzahl Bewerber', 'weibliche\nBewerbende':'darunter Frauen'}, axis=1, inplace=True)
file_sk_2019.drop('Unnamed: 3', axis=1, inplace=True)
file_sk_2019['city'] = file_sk_2019['city'].str.replace('51-BRB', 'Brandeburg an der Havel').str.replace('52-CB', 'Cottbus')
file_sk_2019['city'] = file_sk_2019['city'].str.replace('53-FF', 'Frankfurt (Oder)').str.replace('54-P', 'Potsdam')

In [35]:
file_sk = file_sk_2014.append(file_sk_2019)

In [36]:
full_table_bb = file_sk.append(full_table_bb)

In [37]:
full_table_bb = full_table_bb[full_table_bb['Wg_o_P'] == 'P']
full_table_bb.drop(['Personen mit Unionsbürger-schaft', 'dar. Personen mit Unionsbürgerschaft'], axis=1, inplace=True)
full_table_bb.reset_index(drop=True, inplace=True)
full_table_bb.drop(['Wg_o_P'], axis=1, inplace=True)

In [38]:
full_table_bb['state'] = 'BB'

In [39]:
dupl= agg_sheet_14.groupby('city', as_index=False).count()
dupl[dupl['city_id_bb'] != 1]
agg_sheet_14 = agg_sheet_14[(agg_sheet_14['city'] != 'Golzow')]
agg_sheet_14 = agg_sheet_14[~agg_sheet_14['city'].astype(str).str.contains('Mittenwalde')]
agg_sheet_14.dropna(inplace=True)
agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Baruth/Mark',
                                                        'Baruth-Mark').str.replace('Dahme/Mark',
                                                                                   'Dahme-Mark').str.replace('Falkenberg/Elster', 
                                                                                                            'Falkenberg (Elster)')
agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Fürstenberg/Havel',
                                                        'Fürstenberg (Havel)').str.replace('Fürstenwalde/Spree', 
                                                                                           'Fürstenwalde-Spree').str.replace('Glienicke/Nordbahn',
                                                                                                                            'Glienicke-Nordbahn')
agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Herzberg/Elster',
                                                        'Herzberg(Elster)').str.replace('Jabobsdorf', 
                                                                                           'Jacobsdorf').str.replace('Ketzin/Havel',
                                                                                                                            'Ketzin')
agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Legde/Quitzöbel',
                                                        'Legde-Quitzöbel').str.replace('Löwenberger Land ', 
                                                                                           'Löwenberger Land').str.replace('Lübben (Spreewald)',
                                                                                                                            'Lübben')
agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Lübbenau/Spreewald',
                                                        'Lübbenau-Spreewald').str.replace('Mühlberg/Elbe', 
                                                                                           'Mühlberg(Elbe)').str.replace('Neuhausen/Spree',
                                                                                                                            'Neuhausen-Spree')

agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Paulinenaue',
                                                        'Paulineaue').str.replace('Petershagen/Eggersdorf', 
                                                                                  'Petershagen-Eggersdorf').str.replace('Plantetal',
                                                                                                                        'Planetal')

agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Schwedt/Oder',
                                                        'Schwedt-Oder').str.replace('Vetschau/Spreewald', 
                                                                                  'Vetschau-Spreewald').str.replace('Wiesenburg/Mark',
                                                                                                                        'Wiesenburg-Mark')

agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Wittstock/Dosse',
                                                        'Wittstock-Dosse').str.replace('Wusterhausen/Dosse', 
                                                                                  'Wusterhausen-Dosse')

agg_sheet_14['city'] = agg_sheet_14['city'].str.replace('Bernau bei Berlin', 'Bernau')
full_table_bb.loc[full_table_bb['city'].str.contains('Jabobsdorf'), 'city'] = 'Jacobsdorf'
full_table_bb.loc[full_table_bb['city'].str.contains('Brieskow-Finkenherd'), 'city'] = 'Brieskow-Finkenheerd'
full_table_bb.loc[full_table_bb['city'].str.contains('Plantetal'), 'city'] = 'Planetal'
full_table_bb.loc[full_table_bb['city'].str.contains('Bernau bei Berlin'), 'city'] = 'Bernau'

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version.


In [40]:
full_table_bb = pd.merge(full_table_bb, agg_sheet_14, on='city', how='left', validate='m:1')
full_table_bb.loc[full_table_bb['city'].str.contains('Brandeburg an der Havel'), 'city_id_bb'] = 51000
full_table_bb.loc[full_table_bb['city'].str.contains('Cottbus'), 'city_id_bb'] = 52000
full_table_bb.loc[full_table_bb['city'].str.contains('Frankfurt'), 'city_id_bb'] = 53000
full_table_bb.loc[full_table_bb['city'].str.contains('Potsdam'), 'city_id_bb'] = 54000
full_table_bb['city_id_bb'] = full_table_bb['city_id_bb'].astype(str).str.replace(r'\s', '').astype(float)

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [41]:
data_org_bb = data_org[data_org['Land'] == 12]
data_org_bb['city_id_bb'] = data_org_bb['3'].astype(int).astype(str) + data_org_bb['5']
data_org_bb['city_id_bb'] = data_org_bb['city_id_bb'].astype(float)
data_org_bb = data_org_bb[['city_id_bb', 'city_id', 'plz']]

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
data_bb = pd.merge(full_table_bb, data_org_bb[['city_id_bb', 'plz', 'city_id']], on='city_id_bb', how='outer', validate='m:1')
data_bb[data_bb['city_id_bb'].isna()]

,party_long,party,Anzahl Bewerber,darunter Frauen,city,year,state,city_id_bb,plz,city_id
220,Sozialdemokratische Partei Deutschlands,SPD,13.0,3.0,Lübben,2014.0,BB,NaN,<NA>,NaN
221,DIE LINKE,DIE LINKE,6.0,0.0,Lübben,2014.0,BB,NaN,<NA>,NaN
222,Christlich Demokratische Union Deutschlands,CDU,10.0,1.0,Lübben,2014.0,BB,NaN,<NA>,NaN
223,Freie Demokratische Partei,FDP,3.0,0.0,Lübben,2014.0,BB,NaN,<NA>,NaN
224,BÜNDNIS 90/DIE GRÜNEN,GRÜNE/B90,4.0,2.0,Lübben,2014.0,BB,NaN,<NA>,NaN
225,Sozialdemokratische Partei Deutschlands,SPD,7.0,1.0,Mittenwalde,2014.0,BB,NaN,<NA>,NaN
226,DIE LINKE,DIE LINKE,11.0,3.0,Mittenwalde,2014.0,BB,NaN,<NA>,NaN
227,Christlich Demokratische Union Deutschlands,CDU,21.0,3.0,Mittenwalde,2014.0,BB,NaN,<NA>,NaN
228,Alternative für Deutschland,AfD,2.0,0.0,Mittenwalde,2014.0,BB,NaN,<NA>,NaN
229,Christlich Demokratische Union Deutschlands,CDU,1.0,0.0,Golzow,2014.0,BB,NaN,<NA>,NaN


In [43]:
data_so_far = data_so_far.append(data_bb)

In [51]:
data_so_far.drop('bl_kuerzel', axis=1)
data_so_far

,city,party,Anzahl Bewerber,darunter Frauen,year,state,plz,city_id,Land,bl_kuerzel
0,Stuttgart,AfD,60,16,2019.0,BW,70173,8111000.0,8.0,BW
1,Stuttgart,CDU,60,25,2019.0,BW,70173,8111000.0,8.0,BW
2,Stuttgart,DIE LINKE,60,29,2019.0,BW,70173,8111000.0,8.0,BW
3,Stuttgart,FDP,60,18,2019.0,BW,70173,8111000.0,8.0,BW
4,Stuttgart,GRÜNE,60,30,2019.0,BW,70173,8111000.0,8.0,BW
...,...,...,...,...,...,...,...,...,...,...
19079,NaN,NaN,NaN,NaN,NaN,NaN,17291,12073490.0,NaN,NaN
19080,NaN,NaN,NaN,NaN,NaN,NaN,17291,12073520.0,NaN,NaN
19081,NaN,NaN,NaN,NaN,NaN,NaN,16306,12073309.0,NaN,NaN
19082,NaN,NaN,NaN,NaN,NaN,NaN,17268,12073157.0,NaN,NaN


In [45]:
data_so_far.drop(['Prozent Frauen', 'party_long', 'city_id_bb'], axis=1, inplace=True)
data_so_far.reset_index(drop=True, inplace=True)
data_so_far['party'] = data_so_far['party'].str.replace('GRÜNE/B 90', 'GRÜNE')

In [47]:
# standardize city names for merging w/ crime data
data_so_far['city'] = data_so_far['city'].str.split(',', expand=True)[0]
data_so_far['city'] = data_so_far['city'].str.replace('ä', 'a', re.IGNORECASE).str.replace('ö','o', re.IGNORECASE).str.replace('ü', 'u', re.IGNORECASE).str.replace('ß', 'ss', re.IGNORECASE)

In [46]:
data_so_far

,city,party,Anzahl Bewerber,darunter Frauen,year,state,plz,city_id,Land,bl_kuerzel
0,"Stuttgart, Landeshauptstadt",AfD,60,16,2019.0,BW,70173,8111000.0,8.0,BW
1,"Stuttgart, Landeshauptstadt",CDU,60,25,2019.0,BW,70173,8111000.0,8.0,BW
2,"Stuttgart, Landeshauptstadt",DIE LINKE,60,29,2019.0,BW,70173,8111000.0,8.0,BW
3,"Stuttgart, Landeshauptstadt",FDP,60,18,2019.0,BW,70173,8111000.0,8.0,BW
4,"Stuttgart, Landeshauptstadt",GRÜNE,60,30,2019.0,BW,70173,8111000.0,8.0,BW
...,...,...,...,...,...,...,...,...,...,...
19079,NaN,NaN,NaN,NaN,NaN,NaN,17291,12073490.0,NaN,NaN
19080,NaN,NaN,NaN,NaN,NaN,NaN,17291,12073520.0,NaN,NaN
19081,NaN,NaN,NaN,NaN,NaN,NaN,16306,12073309.0,NaN,NaN
19082,NaN,NaN,NaN,NaN,NaN,NaN,17268,12073157.0,NaN,NaN


In [307]:
data_so_far.to_csv(r'C:\Users\mariu\Documents\pol_viol\pol_viol\election_data_edited.csv')

In [ ]:
# merge control variables here!